In [ ]:
import os
import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
from PIL import Image

import torch
from torchvision import transforms
import timm
from ultralytics import YOLO

from tqdm import tqdm


In [2]:
model = YOLO("yolov8n.pt")
model.info()

YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(129, 3157200, 0, 8.8575488)

In [3]:
print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("사용 중인 디바이스:", torch.cuda.get_device_name(0))

CUDA 사용 가능 여부: True
사용 중인 디바이스: NVIDIA GeForce RTX 5080


In [ ]:
results = model.train(
    data='./dataset.yaml',  # 너가 만든 YAML 경로로 수정
    epochs=100,
    batch=16,  # GPU 메모리에 따라 조정
    imgsz=640,
    device=0,
    cache=True
)

In [4]:
model = YOLO("./best.pt")

train_results = model.predict(
    source="./test_images",     # 테스트 이미지 디렉토리
    conf=0.3,                  # confidence threshold
    imgsz=640,                  # 이미지 크기
    save=True,                  # 예측 결과 이미지 저장
    save_txt=True,              # 바운딩 박스 txt 저장
    save_conf=True,             # confidence 점수까지 포함
    show_labels=True,           # 이미지에 라벨 표시
    show_conf=True,             # confidence score 표시
    name="predict"              # 저장 디렉토리 이름
)



image 1/843 c:\Users\dltmd\OneDrive\Desktop\testing\additional_data\test_images\1.png: 640x512 1  5mg, 1  800mg, 1  10mg, 1 , 33.8ms
image 2/843 c:\Users\dltmd\OneDrive\Desktop\testing\additional_data\test_images\10.png: 640x512 1  5mg, 1  100mg, 1  20mg, 1 , 9.7ms
image 3/843 c:\Users\dltmd\OneDrive\Desktop\testing\additional_data\test_images\100.png: 640x512 1  5mg, 1  100mg, 1  30mg, 1 , 8.5ms
image 4/843 c:\Users\dltmd\OneDrive\Desktop\testing\additional_data\test_images\1003.png: 640x512 1 ()(), 1  20mg, 1  40/5mg, 1  50/1000mg, 9.3ms
image 5/843 c:\Users\dltmd\OneDrive\Desktop\testing\additional_data\test_images\1004.png: 640x512 1 ()(), 1  20mg, 1  40/5mg, 1  50/1000mg, 9.8ms
image 6/843 c:\Users\dltmd\OneDrive\Desktop\testing\additional_data\test_images\1005.png: 640x512 1 ()(), 1  20mg, 1  40/5mg, 1  50/1000mg, 9.3ms
image 7/843 c:\Users\dltmd\OneDrive\Desktop\testing\additional_data\test_images\1006.png: 640x512 1 ()(), 1  75mg, 1  50/850mg, 1  5/100mg, 8.7ms
image 8/843 c:\

In [ ]:
# 설정
CONF_THRESHOLD = 0.82
NUM_CLASSES = 73
TOL = 0.01
label_dir = "./runs/detect/predict/labels"
image_dir = "./test_images"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 로드
eff_model = timm.create_model('efficientnetv2_rw_s', pretrained=False, num_classes=NUM_CLASSES)
eff_model.load_state_dict(torch.load("./best_model.pth", map_location=device))
eff_model = eff_model.to(device)
eff_model.eval()

# 전처리
eff_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# 분류 함수
def classify_with_effnet(image, xyxy):
    x1, y1, x2, y2 = map(int, xyxy)
    crop = image[y1:y2, x1:x2]
    if crop.size == 0:
        return None
    pil_crop = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
    tensor = eff_transform(pil_crop).unsqueeze(0).to(device)
    with torch.no_grad():
        out = eff_model(tensor)
        pred = out.argmax(1).item()
    return pred

# 라벨 파일 처리
for label_file in os.listdir(label_dir):
    if not label_file.endswith(".txt"):
        continue

    image_id = Path(label_file).stem
    label_path = os.path.join(label_dir, label_file)
    image_path = os.path.join(image_dir, f"{image_id}.jpg")
    if not os.path.exists(image_path):
        image_path = os.path.join(image_dir, f"{image_id}.png")
    if not os.path.exists(image_path):
        print(f"이미지 없음: {image_id}")
        continue

    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지 로딩 실패: {image_path}")
        continue

    with open(label_path, "r") as f:
        lines = f.readlines()

    updated = False
    result_boxes = []  # (class, cx, cy, w, h, conf)

    for line_num, line in enumerate(lines):
        parts = line.strip().split()
        if len(parts) != 6:
            continue

        class_id, cx, cy, w, h, conf = parts
        cx, cy, w, h, conf = map(float, [cx, cy, w, h, conf])
        class_id = int(class_id)
        original_class = class_id

        H, W = image.shape[:2]
        cx_pix, cy_pix = cx * W, cy * H
        w_pix = w * W
        h_pix = h * H
        x1 = max(0, int(cx_pix - w_pix / 2))
        y1 = max(0, int(cy_pix - h_pix / 2))
        x2 = min(W, int(cx_pix + w_pix / 2))
        y2 = min(H, int(cy_pix + h_pix / 2))

        # 신뢰도 낮은 경우 재분류
        if conf < CONF_THRESHOLD:
            new_class = classify_with_effnet(image, (x1, y1, x2, y2))
            if new_class is not None:
                # 중복 검사 (기존 결과 리스트에서)
                is_duplicate = False
                for c, ecx, ecy, ew, eh, _ in result_boxes:
                    if (c == new_class and
                        abs(cx - ecx) < TOL and abs(cy - ecy) < TOL and
                        abs(w - ew) < TOL and abs(h - eh) < TOL):
                        is_duplicate = True
                        break

                if is_duplicate:
                    print(f"🗑️ 중복으로 제거됨 → 이미지 {image_id}, 라인 {line_num}, class {new_class}")
                    continue
                else:
                    print(f"🔄 이미지 {image_id}, 라인 {line_num}: class {original_class} → {new_class}, conf {conf:.2f} → 1.00")
                    class_id = new_class
                    conf = 1.00
                    updated = True

        result_boxes.append((class_id, cx, cy, w, h, conf))

    # 저장
    with open(label_path, "w") as f:
        for c, cx, cy, w, h, conf in result_boxes:
            f.write(f"{c} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f} {conf:.2f}\n")

    if updated:
        print(f"\t✅ '{label_file}' 수정 완료.")


🔄 이미지 1027, 라인 2: class 20 → 20, conf 0.41 → 1.00
🗑️ 중복으로 제거됨 → 이미지 1027, 라인 3, class 20
✅ '1027.txt' 수정 완료.
🔄 이미지 1273, 라인 3: class 10 → 18, conf 0.73 → 1.00
✅ '1273.txt' 수정 완료.
🔄 이미지 13, 라인 2: class 20 → 20, conf 0.51 → 1.00
✅ '13.txt' 수정 완료.
🔄 이미지 1418, 라인 3: class 55 → 45, conf 0.47 → 1.00
✅ '1418.txt' 수정 완료.
🔄 이미지 409, 라인 3: class 67 → 65, conf 0.40 → 1.00
✅ '409.txt' 수정 완료.
🔄 이미지 411, 라인 3: class 45 → 65, conf 0.47 → 1.00
✅ '411.txt' 수정 완료.
🗑️ 중복으로 제거됨 → 이미지 507, 라인 3, class 60
🗑️ 중복으로 제거됨 → 이미지 623, 라인 3, class 38
🔄 이미지 747, 라인 3: class 71 → 71, conf 0.49 → 1.00
✅ '747.txt' 수정 완료.


In [ ]:
# 1. class_id → category_id 매핑 로드
with open("classid_to_categoryid.json", "r", encoding="utf-8") as f:
    classid_to_categoryid = json.load(f)

# 2. 경로 설정
label_dir = "./runs/detect/predict/labels"
image_dir = "./test_images"  # 이미지 크기 참조용

# 3. 결과 저장 리스트
csv_rows = []
annotation_id = 1

# 4. 라벨 파일 순회
for label_file in sorted(os.listdir(label_dir), key=lambda x: int(Path(x).stem)):
    if not label_file.endswith(".txt"):
        continue

    image_id = int(Path(label_file).stem)
    label_path = os.path.join(label_dir, label_file)
    image_path_jpg = os.path.join(image_dir, f"{image_id}.jpg")
    image_path_png = os.path.join(image_dir, f"{image_id}.png")

    # 이미지 크기 가져오기
    import cv2
    if os.path.exists(image_path_jpg):
        image = cv2.imread(image_path_jpg)
    elif os.path.exists(image_path_png):
        image = cv2.imread(image_path_png)
    else:
        print(f"⚠️ 이미지 {image_id} 없음. 건너뜀.")
        continue

    H, W = image.shape[:2]

    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 6:
                continue

            class_id, cx, cy, w, h, conf = parts
            class_id = int(class_id)
            if str(class_id) not in classid_to_categoryid:
                print(f"⚠️ class_id {class_id} 매핑 없음. 건너뜀.")
                continue
            category_id = int(classid_to_categoryid[str(class_id)])

            cx, cy, w, h, conf = map(float, [cx, cy, w, h, conf])
            bbox_x = int((cx - w / 2) * W)
            bbox_y = int((cy - h / 2) * H)
            bbox_w = int(w * W)
            bbox_h = int(h * H)

            csv_rows.append([
                annotation_id,
                image_id,
                category_id,
                bbox_x,
                bbox_y,
                bbox_w,
                bbox_h,
                round(conf, 2)
            ])
            annotation_id += 1

# 5. DataFrame 저장
df = pd.DataFrame(csv_rows, columns=[
    "annotation_id", "image_id", "category_id",
    "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"
])
df.to_csv("submission_formatted.csv", index=False)

print("제출용 CSV 저장 완료: submission_formatted.csv")


✅ 제출용 CSV 저장 완료: submission_formatted.csv
